In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[:5])

up_89


In [1]:
import os
from pprint import pprint

# LangChain 모듈 임포트
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# Upstage 모듈 임포트
# (주의: 이 모듈을 사용하려면 'langchain-upstage' 패키지가 설치되어 있어야 합니다.)
from langchain_upstage import ChatUpstage, UpstageEmbeddings

# ----------------------------------------------------------------------
# 0단계: 필수 설정 및 파일 로드
# ----------------------------------------------------------------------

# Upstage API 키 확인 (환경 변수 UPSTAGE_API_KEY에 설정해야 합니다)
if not os.getenv("UPSTAGE_API_KEY"):
    # 주피터 노트북 환경에서는 여기에 API 키를 직접 설정할 수 있습니다.
    # os.environ["UPSTAGE_API_KEY"] = "여기에_당신의_API_키를_입력하세요"
    try:
        if not os.getenv("UPSTAGE_API_KEY"):
             raise ValueError("UPSTAGE_API_KEY 환경 변수가 설정되지 않았습니다.")
    except ValueError as e:
        print(f"오류: {e}")
        # API 키가 없으면 나머지 코드를 실행할 수 없으므로 강제 종료
        raise SystemExit("API 키를 설정하고 다시 시도하세요.")
        

# 문서 파일 경로 (주피터 노트북 파일 위치 기준 상대 경로)
DOCUMENT_PATH = '../data/콘텐츠분쟁해결_사례.pdf'
print(f"로드할 문서 경로: {DOCUMENT_PATH}")

# 문서 로드
print(">>> 0단계: 문서 로드 중...")
try:
    loader = PyPDFLoader(DOCUMENT_PATH)
    docs = loader.load()
    print(f"문서 로드 완료. 총 {len(docs)} 페이지 로드됨.")
except Exception as e:
    print(f"문서 로드 실패: {e}. 파일을 올바른 경로에 배치했는지 확인해주세요.")
    raise # 오류 발생 시 실행 중지

# ----------------------------------------------------------------------
# 1단계: 문서 분할 설정 및 실행
# ----------------------------------------------------------------------
print("\n>>> 1단계: 문서 분할 중...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
split_docs = text_splitter.split_documents(docs)
print(f"문서 분할 완료. 총 {len(split_docs)}개의 청크 생성됨.")


# ----------------------------------------------------------------------
# 2단계: 임베딩 및 벡터 저장소 설정
# ----------------------------------------------------------------------
print("\n>>> 2단계: 임베딩 모델 및 벡터 저장소 설정 중...")
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents(split_docs, embeddings)
print("벡터 저장소(FAISS) 생성 완료.")


# ----------------------------------------------------------------------
# 3단계: 검색기 설정
# ----------------------------------------------------------------------
print("\n>>> 3단계: 검색기(Retriever) 설정 중...")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # 상위 5개 관련 사례 검색
)


# ----------------------------------------------------------------------
# 4단계, 5단계: LLM 및 프롬프트 설정
# ----------------------------------------------------------------------
print("\n>>> 4, 5단계: LLM(Solar-Pro) 및 법률 자문 프롬프트 설정 중...")
# 4단계: LLM 설정
llm = ChatUpstage(
    model="solar-pro",
    base_url="https://api.upstage.ai/v1",
    temperature=0.2
)

# 5단계: 법률 자문 프롬프트 작성
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


# ----------------------------------------------------------------------
# 6단계: QA 체인 생성
# ----------------------------------------------------------------------
print("\n>>> 6단계: RetrievalQA 체인 생성 중...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("QA 체인 생성 완료.")


# ----------------------------------------------------------------------
# 7단계: 테스트 질문 작성 및 실행
# ----------------------------------------------------------------------
print("\n>>> 7단계: 테스트 질문 실행 중...")
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
]

for i, question in enumerate(test_questions):
    print("=" * 80)
    print(f"[{i+1}/{len(test_questions)}] 상담 내용: {question}")

    # QA 체인 호출
    response = qa_chain.invoke({"query": question})
    
    # 결과 출력
    print("\n--- 전문 법률 조언 ---")
    print(response["result"])
    
    print("\n--- 참조 문서 ---")
    for j, doc in enumerate(response["source_documents"]):
        page_num = doc.metadata.get('page', 'N/A')
        print(f"  [참조 {j+1}] 페이지: {page_num} - 내용 일부: {doc.page_content[:150]}...")

print("\n" + "=" * 80)
print("모든 테스트 질문 실행 완료.")

c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-aEDdjKx7-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로드할 문서 경로: ../data/콘텐츠분쟁해결_사례.pdf
>>> 0단계: 문서 로드 중...
문서 로드 완료. 총 109 페이지 로드됨.

>>> 1단계: 문서 분할 중...
문서 분할 완료. 총 104개의 청크 생성됨.

>>> 2단계: 임베딩 모델 및 벡터 저장소 설정 중...
벡터 저장소(FAISS) 생성 완료.

>>> 3단계: 검색기(Retriever) 설정 중...

>>> 4, 5단계: LLM(Solar-Pro) 및 법률 자문 프롬프트 설정 중...

>>> 6단계: RetrievalQA 체인 생성 중...
QA 체인 생성 완료.

>>> 7단계: 테스트 질문 실행 중...
[1/3] 상담 내용: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

--- 전문 법률 조언 ---
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안

#### 1. **사례 기반 쟁점 분석**  
제시된 사례(2006, 2009년 시스템 오류 사례 등)에 따르면, 게임사의 복구 거부 사유는 주로 다음 두 가지로 요약됩니다:  
- **(1) 계정 명의 불일치**: 게임사가 현금거래 또는 계정 공유를 금지하는 약관을 근거로, 신청인이 계정 명의자가 아닐 경우 복구를 거부하는 경우(2006년 사례).  
- **(2) 시스템 오류 입증 부족**: 게임사가 오류 발생 사실을 인정하지 않거나, 신청인의 사용 기록(예: 정상 로그인 이력)을 근거로 기술적 결함을 부인하는 경우(2009년 사례).  

#### 2. **관련 법령 및 약관 검토**  
- **민법 제250조(도품·유실물 특례)**:  
  - 게임 아이템이 "도품" 또는 "유실물"에 해당할 경우, 피해자(아이템 소유자)는 2년 내 반환을 청구할 수 있으나, **금전성 아이템(예: 게임머니)은 적용되지 않음**.  
  - 단, **민법 제251조**에 따라 신청인이 "선의"로 아이템을 취득했다면, 게임사는 